In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# changing matplotlib the default style
matplotlib.style.use('ggplot')

from PDB_Order_Fixer import PDB_Order_Fixer
import mdtraj as md
import os
import numpy as np
import h5py

import datetime
import glob
import copy
from functools import partial 
import operator
import time

import random 
import subprocess
from subprocess import Popen
import sys
from io_functions import *
from custom_clusterer import *
from custom_tica import *
from custom_featurizer import *
from pdb_editing import *
from analysis import *
from io_functions import *
#from topology_fixing import *
from subsampling import *
from conversions import *
from custom_msm import *
from grids import *
from docking_analysis import *

Couldn't import dot_parser, loading of dot files will not be possible.


/home/enf/software/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1318: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
from operator import itemgetter

In [3]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In this iPython notebook, we will featurize MOR ligand binding simulation by pairwise distances between the ligand and different receptor residues. We will then perform tICA and prospectively build an MSM. 

In [4]:
from detect_intermediates import *
from interpret_tICs import *

we are operating on biox3


In [5]:
from b1ar_feature_types import *
from get_variable_names import *
from b1ar_tica_config import *
from residue import Residue, Atom
ori_feature_name = copy.deepcopy(feature_name)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 

In [6]:
rho = 0.1
rho_string = "0pt1"
lag_time = 5
n_components = 10
n_clusters = 1000

rho = 0.01
rho_string = "0pt01"
lag_time=50
schemes = ["closest-heavy", "CA"]
feature_name = "%s-CA" %ori_feature_name
#feature_name = "%s_phi_psi_chi2" %feature_name
(active_ref_dir, inactive_ref_dir, simulation_ref_dir, scripts_dir,
          ligand_dir, agonist_dir, inverse_agonist_dir, biased_agonist_dir, ref_receptors_dir, whole_trajectory_pnas,
          sasa_file) = get_base_files(base)

tica_dir = get_tica_dir(base, is_sparse, lag_time, n_components, feature_name, 
                                 wolf_string, shrinkage_string, rho_string)
ori_tica_dir = copy.deepcopy(tica_dir)
#tica_dir = "%s-normalized" % ori_tica_dir
features_dir = get_features_dir(base, feature_name)

landmarks_dir = get_landmarks_dir(tica_dir)
analysis_dir = get_analysis_dir(tica_dir, n_clusters, sampling_method)
gmm_dir = get_gmm_dir(tica_dir)
rf_dir = get_rf_dir(tica_dir)


ref_tica_dir, ref_tica_coords = get_ref_tica_dirs(tica_dir)

graph_file = get_graph_file(tica_dir, msm_lag_time, n_clusters)

pnas_titles =  ["tm6_tm3_dist", "rmsd_npxxy_inactive", "rmsd_npxxy_active", "rmsd_connector_inactive", "rmsd_connector_active"]
pnas_features_dir = analysis_dir


(clusterer_dir, msm_model_dir, macrostate_dir, features_known, model_dir, projected_features_dir,
         projection_operator_dir, ktica_fit_model_filename, ktica_projected_data_filename, nystroem_data_filename,
         mutual_information_csv, pearson_csv) = get_tica_files(base, tica_dir, n_clusters, msm_lag_time, n_macrostates)

(standardized_features_dir, feature_residues_csv, feature_residues_pkl,
          contact_csv, ref_features_dir) = get_feature_files(features_dir)

(kmeans_csv, tica_coords_csv, features_csv, active_rmsd_dir, inactive_rmsd_dir, active_pnas_dir, inactive_pnas_joined, active_pnas_joined,
        clusters_map_file, ktica_clusters_map_file, analysis_file, combined_file, docking_summary, docking_joined, docking_z_scores_csv,
        aggregate_docking, aggregate_docking_joined, docking_pnas_joined, aggregate_docking_pnas, aggregate_docking_pnas_joined, docking_multiple_ligands,
        docking_distances_file, docking_pdf, mmgbsa_docking_distances, pnas_coords, mmgbsa_dir, mmgbsa_csv, mmgbsa_pdf, aggregate_mmgbsa,
        aggregate_mmgbsa_joined, aggregate_mmgbsa_pnas_joined, mmgbsa_z_scores_csv, active_clusters_csv, intermediate_clusters_csv,
        inactive_clusters_csv, pnas_clusters_averages, tica_clusters_averages, tica_classes_csv, tica_samples_csv, subgraph_save_base,
        degree_save_base, degree_map_csv, degree_z_map_csv, aggregate_docking_pnas_degree_z_joined, tic_residue_csv, feature_coefs_csv,
        duplicated_feature_coefs_csv) = get_analysis_files(analysis_dir, n_clusters, tica_dir, tica_dir, sampling_method, n_samples, precision,
                                                           msm_lag_time)

(inactive_pnas_distances_dir, active_pnas_distances_dir, active_pnas_all_distances_dir,
          inactive_pnas_distances_new_csv, active_pnas_distances_new_csv, active_pnas_joined, active_pnas_means, pnas_coords_dir,
          pnas_coords_csv, pnas_all_coords_csv, pnas_coords_hexbin_dir, pnas_coords_co_crystallized_docking_dir,
          pnas_coords_active_colors_dir, user_defined_features_file, reaction_coordinates_trajs_file) = get_pnas_files(whole_trajectory_pnas, pnas_features_dir)

features_dir = get_features_dir(base, feature_name)



graph_file = get_graph_file(tica_dir, msm_lag_time, n_clusters)
(scripts_dir, pymol_fixpdb_dir) = get_script_dir(scripts_dir)
(save_dir, reimaged_dir, mae_dir, combined_reimaged_dir, grid_dir, docking_dir) = get_docking_dirs(tica_dir, n_clusters, n_components, n_samples, sampling_method, precision)


/home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA
/home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA


In [16]:
from ipyparallel import Client
rc = Client()
print(len(rc.ids))
dview = rc[:]
dview.map(os.chdir, ['/home/enf/b2ar_analysis/conformation']*len(rc.ids))

134


<AsyncMapResult: chdir>

In [ ]:
import custom_featurizer
reload (custom_featurizer)
from custom_featurizer import *
suf_trajectories = [t for t in get_trajectory_files(traj_dir, ".h5")]
#fix_chain_names(suf_trajectories, dview)
#featurize_contacts_custom(traj_dir, features_dir = features_dir, traj_ext = traj_ext, contact_residue_pairs_file = feature_residues_pkl, structures=[inactive_dir, active_dir], contact_residues=contact_residues,
#                          residues_map = None, contact_cutoff = cutoff, parallel = False, exacycle = exacycle, traj_top_structure = None, iterative=False,
#                          user_specified_atom_residue_pairs = [], load_from_file=False, worker_pool=None, schemes=schemes)

In [ ]:
traj_dir

In [ ]:
import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *
user_defined_features_dir = "%s/user_defined_features" %traj_dir
if not os.path.exists(user_defined_features_dir):
    os.makedirs(user_defined_features_dir)
compute_user_defined_features_wrapper(traj_dir, traj_ext, inactive_dir, active_dir, None,
                                        feature_name_residues_dict, user_defined_features_file, worker_pool=dview, 
                                        save_dir=user_defined_features_dir, parallel=False)

In [ ]:
import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *
featurize_contacts_custom(traj_dir, features_dir = features_dir, traj_ext = traj_ext, contact_residue_pairs_file = feature_residues_pkl, structures=[inactive_dir, active_dir], contact_residues=contact_residues,
                          residues_map = None, contact_cutoff = cutoff, parallel = False, exacycle = exacycle, traj_top_structure = None, iterative=False,
                          user_specified_atom_residue_pairs = [], load_from_file=False, worker_pool=None, schemes=schemes)

In [ ]:
fit_normalizer(features_dir)

In [ ]:
import pickle
with open(feature_residues_pkl, "rb") as f:
    feature_residues = pickle.load(f)

In [ ]:
feature_residues

In [ ]:
import custom_tica
reload(custom_tica)
from custom_tica import *
fit_and_transform(features_directory = features_dir, model_dir = tica_dir, stride=5, lag_time = lag_time, n_components = n_components, sparse = sparse, wolf = wolf, rho = rho, shrinkage = shrinkage, parallel=True, traj_ext = traj_ext, normalize=True, partial_fit=True)

In [ ]:
import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *

featurize_contacts_custom(ref_receptors_dir, features_dir = ref_features_dir, traj_ext = ".pdb", contact_residue_pairs_file = feature_residues_pkl, structures=[inactive_dir, active_dir], contact_residues=contact_residues,
                          residues_map = None, contact_cutoff = cutoff, parallel = False, exacycle = exacycle, traj_top_structure = None, iterative=False,
                          user_specified_atom_residue_pairs = [], load_from_file=False, worker_pool=None, schemes=schemes)

In [9]:
ref_receptors_dir

'/home/enf/md_simulations/B1AR/reference_receptors'

In [ ]:
transform(existing_model = projection_operator_dir, features_directory = ref_features_dir, tica_dir = ref_tica_dir, normalizer=verboseload("%s/normalizer.h5" %features_dir))


In [ ]:
import custom_tica
reload(custom_tica)
from custom_tica import *
transform_to_h5(projection_operator_dir, features_dir, projected_features_dir, worker_pool=None, parallel=True, normalizer=verboseload("%s/normalizer.h5" %features_dir))

In [8]:
tica_object = verboseload(projection_operator_dir)

loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/phi_psi_chi2_allprot_tica_coords.h5"...


In [11]:
#all_tica = verboseload(projected_features_dir)
#all_tica = [x[:,range(1,10)] for x in all_tica]
#print(all_tica[0].shape)
#print(len(all_tica))
#verbosedump(all_tica, projected_features_dir)


In [12]:
#n_components=9

In [9]:
tica_object.timescales_

array([ 1807.27577207,   930.82557327,   715.12014423,   577.5271037 ,
         552.90096526,   471.41700734,   429.5572142 ,   418.77614562,
         389.1365181 ,   374.5275853 ])

In [ ]:
plot_columns(tica_dir, projected_features_dir, titles = ["tIC%d" %j for j in range(1,11)], tICA = True, scale = 1.0, refcoords_file = None)

In [10]:
feature_indices = []
traj_basenames = get_trajectory_files(traj_dir, ".h5")
traj_basenames = [os.path.splitext(os.path.basename(t))[0] for t in traj_basenames]
feature_basenames = get_trajectory_files(features_dir, ".dataset")
feature_basenames = [os.path.splitext(os.path.basename(t))[0] for t in feature_basenames]
for i, traj_basename in enumerate(traj_basenames):
    if traj_basename in feature_basenames:
        feature_indices.append(int(i))

tica_coords = verboseload(projected_features_dir)
pnas_coords = verboseload(user_defined_features_file)
pnas_coords = [pnas_coords[i] for i in feature_indices]
tica_names = ["tIC.%d" %i for i in range(1,n_components+1)]
pnas_names = sorted(feature_name_residues_dict.keys())

loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/phi_psi_chi2_allprot_projected.h5"...
loading "/home/enf/md_simulations/B1AR/all_pnas_features/user_defined_features.h5"...


In [15]:
print(np.concatenate(pnas_coords).shape)
print(np.concatenate(tica_coords).shape)

(29570, 5)
(29570, 10)


In [ ]:
plt.hist(np.concatenate(pnas_coords)[:,0], bins=50)
plt.xlabel("tm6 to tm3 packing")
plt.ylabel("counts")
plt.show()

In [ ]:
from plots import *

In [ ]:
plot_data_vs_data(np.concatenate(tica_coords), np.concatenate(pnas_coords), tica_names, pnas_names, analysis_dir)

In [16]:
subsampled_features_dir = os.path.join(tica_dir, "subsampled_features")
if not os.path.exists(subsampled_features_dir): os.makedirs(subsampled_features_dir)

In [34]:
with open(feature_residues_pkl) as f:
    feature_names = pickle.load(f)
print(feature_names[0].__repr__())

ILE210 to ILE211


In [18]:
ref_features = [load_file(f) for f in get_trajectory_files(ref_features_dir, ".dataset")]
print(len(ref_features))
print(ref_features[0].shape)
print(ref_features[1].shape)
print(len(feature_names))
ref_features = pd.DataFrame(ref_features[0], columns=[str(f) for f in feature_names])

0


IndexError: list index out of range

In [32]:
import interpret_tICs
reload(interpret_tICs)
from interpret_tICs import *
tic_components_dir = tica_dir
important_contact_features = interpret_tIC_components(projection_operator_dir, tic_components_dir, feature_residues_pkl, n_tica_components=n_components, percentile=95)


Interpreting tIC 1
feature_importances_df.shape
(12, 7)
residue_importances_df.shape
(13, 3)
              feature_name      res_i      res_j  resid_i  resid_j  \
5   Thr166_Ca To Ala171_Ca  Thr166_Ca  Ala171_Ca      166      171   
2   Thr166_Ca To Ala169_Ca  Thr166_Ca  Ala169_Ca      166      169   
6   Asn167_Ca To Ala171_Ca  Asn167_Ca  Ala171_Ca      167      171   
7   Trp145_Ca To Thr166_Ca  Trp145_Ca  Thr166_Ca      145      166   
9   Phe164_Ca To Thr166_Ca  Phe164_Ca  Thr166_Ca      164      166   
8   Trp145_Ca To Asp163_Ca  Trp145_Ca  Asp163_Ca      145      163   
11  Val165_Ca To Asn167_Ca  Val165_Ca  Asn167_Ca      165      167   
3   Asn167_Ca To Ala169_Ca  Asn167_Ca  Ala169_Ca      167      169   
4   Phe164_Ca To Tyr170_Ca  Phe164_Ca  Tyr170_Ca      164      170   
0          Phe265 To Leu35     Phe265      Leu35      265       35   
1         Asp147 To Ala169     Asp147     Ala169      147      169   
10  Thr166_Ca To Asn167_Ca  Thr166_Ca  Asn167_Ca      166      167 

In [35]:
import interpret_tICs
reload(interpret_tICs)
from interpret_tICs import *
tic_subsampled_features_file = "%s/features_subsampled.pkl" % tica_dir
subsampled_features_dir = os.path.join(tica_dir, "subsampled_features")
if not os.path.exists(subsampled_features_dir): os.makedirs(subsampled_features_dir)
important_contact_features_pruned, important_contact_features_indices = find_non_zero_features(important_contact_features[0], feature_names)
subsample_features(features_dir, important_contact_features_indices, important_contact_features_pruned, tic_subsampled_features_file)

loading /home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA/RUN0.dataset
loading /home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA/RUN_Active_GP_LigandXAM1.dataset
loading /home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA/RUN_Active_GP_LigandXAM4.dataset
loading /home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA/RUN_R1.dataset
loading /home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA/RUN_R12.dataset
loading /home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA/RUN_Active_GP_LigandXAM7.dataset
loading /home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA/RUN_R15.dataset
loading /home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA/RUN_R4.dataset
loading /home/enf/md_simulations/B1AR/featuresall_residues_5f8u_hom_under_cutoff6A-CA/RUN_R7.dataset
loading /home/enf/md_simulations/B1AR

In [ ]:
protein_top_features = load_file(tic_subsampled_features_file)

In [20]:
from ipyparallel import Client
rc = Client()
dview = rc[:]
dview.map(os.chdir, ['/home/enf/b2ar_analysis/conformation']*len(rc.ids))

print(len(rc.ids))


140


In [ ]:
trajs = []
apo_trajs = []
oxy_trajs = []
suf_trajs = []
trv_trajs = []
trajs = get_trajectory_files(features_dir, ".dataset")
#trajs = [t for t in get_trajectory_files(traj_dir, traj_ext) if "apo_reimaged" not in t and "oxy" not in t]
all_trajs = []
for i, traj in enumerate(trajs):
    all_trajs.append(i)

user_defined_coords = load_file(user_defined_features_file)

            

In [ ]:
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *
n_clusters = 1000
clusterer_dir = "%s/clusterer_%dclusters.h5" %(tica_dir, n_clusters)
if not os.path.exists(clusterer_dir):
    cluster_minikmeans(tica_dir, projected_features_dir, traj_dir, n_clusters=n_clusters, clusterer_dir=clusterer_dir)

In [ ]:
clusterer = verboseload(clusterer_dir)
cluster_labels = clusterer.labels_

In [ ]:
cluster_tica_averages = calculate_cluster_averages_per_feature(clusterer, tica_coords)
cluster_tica_averages = pd.DataFrame(cluster_tica_averages, columns=["tIC.%d" %i for i in range(1, n_components+1)])



In [ ]:
intermediate_actives = cluster_tica_averages.loc[(cluster_tica_averages["tIC.2"] > 0.7) & (cluster_tica_averages["tIC.2"] < 1.2) & (cluster_tica_averages["tIC.5"] > 0.) & (cluster_tica_averages["tIC.2"] < 0.8)]
intermediate_actives

In [ ]:
import custom_msm
reload(custom_msm)
from custom_msm import *
from msm_resampled import *

In [ ]:
clusters_map = make_clusters_map(clusterer)

In [ ]:

plt.clf()
plot_timescales(clusterer_dir, n_clusters, tica_dir, main="MSM Timescales", lag_times=list(range(1,31,6)))

In [ ]:
import msmbui

In [ ]:
lag_time=1
prior_counts = 0.0
msm_dir = os.path.join(tica_dir, "msm_lag-time%d_prior-counts%s_clusters%d.h5" %(lag_time, str(prior_counts), n_clusters))
if not os.path.exists(msm_dir):
    msm_object = build_msm(clusterer_dir, lag_time, msm_dir, prior_counts)
else:
    msm_object = verboseload(msm_dir)

import msm_resampled
reload(msm_resampled)
from msm_resampled import *
total_samples = 20000
num_trajs = len(trajs)
prior_counts = 0.
resampled_traj_to_frames_file = os.path.join(tica_dir, "msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(lag_time, str(prior_counts), n_clusters, total_samples))
#if not os.path.exists(resampled_traj_to_frames_file):
resampled_traj_to_frames = resample_by_msm(total_samples, msm_object, clusters_map, num_trajs, resampled_traj_to_frames_file)
#else:
resampled_traj_to_frames = verboseload(resampled_traj_to_frames_file)

pnas = pnas_coords
pnas_resampled_file = os.path.join(tica_dir, "pnas_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(pnas, resampled_traj_to_frames, pnas_resampled_file)
pnas_resampled = verboseload(pnas_resampled_file)

tica_concatenated = np.concatenate(verboseload(projected_features_dir))
tica_axes = [(np.min(tica_concatenated[:,i]), np.max(tica_concatenated[:,i])) for i in range(0, np.shape(tica_concatenated)[1])]

tica = tica_coords
tica_resampled_file = os.path.join(tica_dir, "tica_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(tica, resampled_traj_to_frames, tica_resampled_file)
tica_resampled = verboseload(tica_resampled_file)

clusterer = verboseload(clusterer_dir)
msm_object = verboseload(msm_dir)

cluster_id_proportion_tuples = []

equilibrium_populations = msm_object.populations_

for cluster_id in msm_object.mapping_.keys():
    state_id = msm_object.mapping_[cluster_id]
    cluster_id_proportion_tuples.append((cluster_id,equilibrium_populations[state_id]))
cluster_id_proportion_tuples = sorted(cluster_id_proportion_tuples, key=itemgetter(1), reverse=True)

#plot_all_tics_and_clusters(tica_dir, tica_resampled_file, clusterer_file, lag_time, tic_range=range(0,n_components), main = "MSM_Reweighted", label = "cluster_id", active_cluster_ids = [c[0] for c in cluster_id_proportion_tuples[0:100]], intermediate_cluster_ids = [], inactive_cluster_ids = [], concatenate=False, axes=tica_axes)

#plot_columns(tica_dir, tica_resampled_file, titles = ["tIC.%d" %i for i in range(1,n_components+1)], main = "mOR_MSM_", tICA = False, scale = 1.0, refcoords_file = ref_tica_coords, axes=tica_axes, concatenate=False, reshape=False)

In [ ]:
graph_file = "%s/msm_lag_time%d_graph.graphml" % (tica_dir, lag_time)
tica_averages = calculate_cluster_averages_per_feature(clusterer, tica)
cluster_attributes = {}
for i, title in enumerate(["tIC.%d" %i for i in range(1,n_components+1)]):
    cluster_attributes[title] = tica_averages[:,i]
construct_graph(msm_dir, clusterer_file, n_clusters, 5, 5, graph_file, inactive = None, active = None, pnas_clusters_averages = None, tica_clusters_averages = None, docking=None, macrostate = None, cluster_attributes=cluster_attributes)

In [ ]:
plot_columns(tica_dir, apo_pnas_resampled_file, titles = sorted(feature_name_residues_dict.keys()), main = "mOR_Apo_MSM_", tICA = False, scale = 1.0, refcoords_file = None, axes=None, concatenate=False, reshape=True)

In [ ]:
lag_time=25
prior_counts = 0.0
msm_dir = os.path.join(tica_dir, "msm_lag-time%d_prior-counts%s_clusters%d.h5" %(lag_time, str(prior_counts), n_clusters))
if not os.path.exists(msm_dir):
    msm = build_msm(clusterer_file, lag_time, msm_dir, prior_counts)
else:
    msm = verboseload(msm_dir)
msm.timescales_

In [ ]:
total_samples = 20000
num_trajs = len(trajs)
resampled_traj_to_frames_file = os.path.join(tica_dir, "msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(lag_time, str(prior_counts), n_clusters, total_samples))
if not os.path.exists(resampled_traj_to_frames_file):
    resampled_traj_to_frames = resample_by_msm(total_samples, msm, clusters_map, num_trajs, resampled_traj_to_frames_file)
else:
    resampled_traj_to_frames = verboseload(resampled_traj_to_frames_file)

In [ ]:
resampled_traj_to_frames_file

In [ ]:
pnas = pnas_coords
pnas_resampled_file = os.path.join(tica_dir, "pnas_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(pnas, resampled_traj_to_frames, pnas_resampled_file)
pnas_resampled = verboseload(pnas_resampled_file)

In [ ]:
tica = tica_coords
tica_resampled_file = os.path.join(tica_dir, "tica_msm_lag-time%d_prior-counts%s_clusters%d_resampled_%d.h5" %(lag_time, str(prior_counts), n_clusters, total_samples))
resample_features_by_msm_equilibirum_pop(tica, resampled_traj_to_frames, tica_resampled_file)
tica_resampled = verboseload(tica_resampled_file)

In [ ]:
tica_resampled = np.vstack([verboseload(pnas_resampled_file)[::1,]])
mins = np.min(tica_resampled, axis=0)
maxs = np.max(tica_resampled, axis=0)
stds = np.std(tica_resampled, axis=0)
custom_lims = [[mins[i] - 1.*stds[i], maxs[i] + 1.*stds[i]] for i in range(0,len(mins))]

In [ ]:
ref_pnas = load_file("/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5")
ref_pnas = np.hstack([np.array(x).T for x in ref_pnas])

In [ ]:
def get_features(features_dir):
    feature_files = get_trajectory_files(features_dir, ".dataset")
    features = np.vstack([load_file(f) for f in feature_files])
    print(features.shape)
    return features

ref_pp_features = pd.DataFrame(get_features(ref_features_dir), columns=feature_names)

ref_pp_tica = pd.read_csv(ref_tica_coords, header=None).values
#ref_pp_tica.columns = ["mOR tIC.%d" %i for i in range(1,ref_pp_tica.shape[1])]

ref_pnas = load_file("/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/ref_coords.h5")
ref_pnas = np.hstack([np.array(x).T for x in ref_pnas])
print(ref_pnas)

#ref_pnas = pd.DataFrame(ref_pnas, columns=sorted(feature_name_residues_dict.keys()))

#ref_features = pd.concat([ref_pp_features*10., ref_lp_features*10., ref_pp_tica, ref_lp_tica, ref_pnas], axis=1)

In [ ]:
ref_pp_tica

In [ ]:
import jointplot_d3
reload(jointplot_d3)
from jointplot_d3 import *
jointplots(pnas_resampled[::1,:], analysis_dir, titles = sorted(feature_name_residues_dict.keys()), main = "mOR_MSM_", refcoords = ref_pnas, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=20, min_density=None, max_diff=3., custom_lims=custom_lims, n_levels=15, n_pts=200j, parallel=True)

In [ ]:
ref_pnas

In [ ]:
jointplots(apo_pnas_resampled[::1,:], analysis_dir, titles = sorted(feature_name_residues_dict.keys()), main = "mOR_APO_MSM_", refcoords=ref_pnas, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=20, min_density=None, max_diff=3., custom_lims=custom_lims, n_levels=10)

In [ ]:
jointplots(suf_pnas_resampled[::1,:], analysis_dir, titles = sorted(feature_name_residues_dict.keys()), main = "mOR_SUF_MSM_", refcoords=ref_pnas, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=20, min_density=None, max_diff=3., custom_lims=custom_lims, n_levels=10)

In [ ]:
tica_resampled = np.vstack([verboseload(tica_resampled_file)[::1,]])
mins = np.min(tica_resampled, axis=0)
maxs = np.max(tica_resampled, axis=0)
stds = np.std(tica_resampled, axis=0)
custom_lims = [[mins[i] - 1.*stds[i], maxs[i] + 1.*stds[i]] for i in range(0,len(mins))]


In [ ]:
print(custom_lims)
print(custom_lims[3][1])

In [ ]:
import jointplot_d3
reload(jointplot_d3)
from jointplot_d3 import *
refcoords_jointplots_dir = "%s/refcoords_jointplots" %analysis_dir
if not os.path.exists(refcoords_jointplots_dir):
    os.makedirs(refcoords_jointplots_dir)
jointplots(verboseload(tica_resampled_file)[::1,:], refcoords_jointplots_dir, titles = ["tIC%d" %j for j in range(1,n_components+1)], main = "BU72 MSM", refcoords=ref_pp_tica, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=10, min_density=None, max_diff=5., custom_lims=custom_lims, n_pts=200j, n_levels=15)
#jointplots(verboseload(apo_tica_resampled_file)[::1,:], refcoords_jointplots_dir, titles = ["tIC%d" %j for j in range(1,n_components+1)], main = "Apo MSM", refcoords=ref_pp_tica, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=10, min_density=None, max_diff=5., custom_lims=custom_lims, n_pts=200j, n_levels=15)


#jointplots(verboseload(suf_tica_resampled_file)[::1,:], refcoords_jointplots_dir, titles = ["tIC%d" %j for j in range(1,n_components+1)], main = "Sufentanil MSM", refcoords=ref_pp_tica, axes=None, data_j=None, titles_j=None, reshape=False, max_tIC=10, min_density=None, max_diff=5., custom_lims=custom_lims, n_pts=200j, n_levels=15)

In [ ]:
import jointplot_d3
reload(jointplot_d3)
from jointplot_d3 import *
jointplots(verboseload(tica_resampled_file)[::1,:], analysis_dir, titles = ["tIC%d" %j for j in range(1,n_components+1)], main = "BU72 MSM", refcoords=None, axes=None, data_j=pnas_resampled, titles_j=sorted(feature_name_residues_dict.keys()), reshape=False, max_tIC=10, min_density=None, max_diff=5., custom_lims=None)

In [8]:
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *

n_samples = 1
n_clusters=300
samples_dir = "%s/clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)
print(samples_dir)
traj_files = get_trajectory_files(traj_dir, traj_ext)    
sample_from_clusterer(clusterer_file, tica_file, [traj_files[i] for i in trajs], 
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=dview, parallel=False, lig_name="NON")

/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/clusterer_25clusters_1samples_samples_kdtree


In [ ]:
get_trajectory_files(traj_dir, traj_ext)

In [ ]:
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *

n_samples = 5
n_clusters=1000
samples_dir = "%s/apo_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/apo_clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)

sample_from_clusterer(apo_clusterer_file, apo_tica_file, [traj_files[i] for i in apo_trajs],
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=dview, parallel=False, lig_name="LIG")

In [ ]:
traj_files

In [ ]:
feature_name_residues_dict.keys()

In [ ]:

np.shape(apo_pnas_resampled)
msm_dir = os.path.join(tica_dir, "msm_lag-time%d_prior-counts%s_clusters%d_analysis" %(lag_time, str(prior_counts), n_clusters))
if not os.path.exists(msm_dir): os.makedirs(msm_dir)

#plot_columns(msm_dir, apo_pnas_resampled_file, titles = feature_name_residues_dict.keys(), main = "Apo-mOR", tICA = False, scale = 1.0, refcoords_file = None, axes=None, concatenate=False)

In [ ]:
import pip

def install(package):
    pip.main(['install', package])
    
install("moviepy")

In [ ]:
import jointplot_d3
reload(jointplot_d3)
from jointplot_d3 import *

for i, traj in enumerate(get_trajectory_files(traj_dir, traj_ext)):
    if "apo_rep_68." in traj:
        index=i
        break

jointplot(apo_pnas_resampled[::10,[6,3]], "%s/test.pdf" %msm_dir, trajectory=apo_pnas[index][::1,[6,3]], 
          titles=["TM6 to TM3 distance", "RMSD of NPxxY to Inactive"], main = "mOR Activation Pathway", 
          video_file = "%s/apo_68.mp4" %msm_dir, custom_xlim=[4.0, 16.0], custom_ylim= [0.0, 1.0])
#test("%s/test2.mp4" %msm_dir)

In [ ]:
build_msm(clusterer_file, lag_time, msm_model_dir)
build_msm(apo_clusterer_file, lag_time, msm_model_dir)

In [ ]:
tics_1_5_file = os.path.join(tica_dir, "tICs_1_5.h5")
tics = load_file(projected_features_dir)
tics_1_5 = [x[:,range(1,5)] for x in tics]
verbosedump(tics_1_5, tics_1_5_file)

clusterer300_clusters = os.path.join(tica_dir, "clusterer300_clusters_tICs1_5.h5")
cluster_minikmeans(tica_dir, tics_1_5_file, traj_dir, n_clusters=300, clusterer_dir=clusterer300_clusters)

In [17]:

tics_2_3_file = os.path.join(tica_dir, "tICs_2_3.h5")
tics = load_file(projected_features_dir)
tics_2_3 = [x[:,range(2,4)] for x in tics]
verbosedump(tics_2_3, tics_2_3_file)

clusterer25_clusters = os.path.join(tica_dir, "clusterer25_clusters_tICs2_3.h5")
if not os.path.exists(clusterer25_clusters):
    cluster_minikmeans(tica_dir, tics_2_3_file, traj_dir, n_clusters=25, clusterer_dir=clusterer25_clusters)

n_samples = 1
n_clusters = 25

traj_files = get_trajectory_files(traj_dir, traj_ext)
featurized_trajectories = [traj_files[i] for i in feature_indices]
traj_files = [t for t in featurized_trajectories if "bu72" in t]
reduced_clusterer = verboseload(clusterer25_clusters)
reduced_clusterer_file = "%s/reduced_clusterer.h5" %tica_dir

reduced_tica_file = "%s/reduced_tica.h5" %tica_dir
reduced_tica = [x[:,[2,3]] for x in tica_coords]
reseeded_dir = "%s/reseeded_samples" %tica_dir
if not os.path.exists(reseeded_dir):
    os.makedirs(reseeded_dir)
verbosedump(reduced_tica, reduced_tica_file)

reduced_clusters = reduced_clusterer.labels_
reduced_clusterer = copy.deepcopy(reduced_clusterer)
reduced_clusterer.labels_ = reduced_clusters
verbosedump(reduced_clusterer, reduced_clusterer_file)
#reseed_from_clusterer(reduced_clusterer_file, "bu72", reseeded_dir, reduced_tica_file, traj_files)


samples_dir = "%s/clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)


reseed_dir = "%s/clusterer_%dclusters_%dsamples_samples_kdtree_reseed" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)
if not os.path.exists(reseed_dir): os.makedirs(reseed_dir)
print(samples_dir)
traj_files = get_trajectory_files(traj_dir, traj_ext)    
#sample_from_clusterer(reduced_clusterer_file, reduced_tica_file, traj_files, 
#                      n_samples, samples_dir, samples_indices_file,
#                      worker_pool=None, parallel=True, lig_name="NON", reseed_dir = reseed_dir)

loading /home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/phi_psi_chi2_allprot_projected.h5
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/phi_psi_chi2_allprot_projected.h5"...
Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/tICs_2_3.h5"... (<type 'list'>)
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/clusterer25_clusters_tICs2_3.h5"...
Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/reduced_tica.h5"... (<type 'list'>)
Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_h

In [29]:
grid_center = "19., 4.5, -9.3"
n_samples=1
active_ref_dir = active_dir

precision = "XP"
docking_dir = "%s/docking_%s_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, precision, n_clusters, n_samples)
if not os.path.exists(docking_dir):
    os.makedirs(docking_dir)
grid_dir = "%s/grids_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(grid_dir):
    os.makedirs(grid_dir)

indices = [0, n_clusters]
chosen_receptors = []
for i in range(indices[0], indices[1]):
  for j in range(0, n_samples):
    chosen_receptors.append("cluster%d_sample%d" %(i, j))

reimaged_dir = samples_dir
mae_dir = reimaged_dir
#remove_ter(reimaged_dir)
#reorder(reimaged_dir)

all_ligands_dir = "/home/enf/b2ar_analysis/all_ligands"


#agonist_ligands = [a for a in agonist_ligands if "TA" not in a]

pprep(mae_dir, ref = active_ref_dir, chosen_receptors = chosen_receptors, worker_pool=None, parallel=True)
generate_grids(mae_dir, grid_center, grid_dir, remove_lig = "LIG", chosen_receptors = chosen_receptors, worker_pool=None, outer_box=30.)


dock_ligands_and_receptors(grid_dir, docking_dir, all_ligands_dir, precision = precision, ext = "-out.maegz", chosen_ligands = False, chosen_receptors = False, parallel = None, grid_ext = ".zip", worker_pool=dview)

#dock_ligands_and_receptors(grid_dir, docking_dir,  biased_agonist_dir, precision = precision, ext = "-out.maegz", chosen_ligands = biased_ligands, chosen_receptors = chosen_receptors, parallel = None, grid_ext = ".grd", worker_pool=dview)
#dock_ligands_and_receptors(grid_dir, docking_dir, agonist_dir, precision = precision, ext = "-out.maegz", chosen_ligands = agonist_ligands, chosen_receptors = chosen_receptors, parallel = None, grid_ext = ".grd", worker_pool=dview)




25
['/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/clusterer_25clusters_1samples_samples_kdtree/cluster0_sample0.pdb', '/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/clusterer_25clusters_1samples_samples_kdtree/cluster10_sample0.pdb', '/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/clusterer_25clusters_1samples_samples_kdtree/cluster11_sample0.pdb']
Done prepping proteins
/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/grids_clusterer_25clusters_1samples_samples_kdtree
already prepped and mae'd protein
already prepped and mae'd protein
already prepped and mae'd protein
already prepped and mae'd protein
already prepped and m

In [30]:
grid_center = "19., 4.5, -9.3"
n_samples=1
active_ref_dir = active_dir

precision = "XP"
docking_dir = "%s/docking_%s_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, precision, n_clusters, n_samples)
if not os.path.exists(docking_dir):
    os.makedirs(docking_dir)
grid_dir = "%s/grids_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(grid_dir):
    os.makedirs(grid_dir)

indices = [0, n_clusters]
chosen_receptors = []
for i in range(indices[0], indices[1]):
  for j in range(0, n_samples):
    chosen_receptors.append("cluster%d_sample%d" %(i, j))

reimaged_dir = samples_dir
mae_dir = reimaged_dir
#remove_ter(reimaged_dir)
#reorder(reimaged_dir)

all_ligands_dir = "/home/enf/b2ar_analysis/all_ligands"


#agonist_ligands = [a for a in agonist_ligands if "TA" not in a]

pprep(mae_dir, ref = active_ref_dir, chosen_receptors = chosen_receptors, worker_pool=None, parallel=True)
generate_grids(mae_dir, grid_center, grid_dir, remove_lig = "LIG", chosen_receptors = chosen_receptors, worker_pool=None, outer_box=30.)


dock_ligands_and_receptors(grid_dir, docking_dir, all_ligands_dir, precision = precision, ext = "-out.maegz", chosen_ligands = False, chosen_receptors = False, parallel = None, grid_ext = ".zip", worker_pool=dview)

#dock_ligands_and_receptors(grid_dir, docking_dir,  biased_agonist_dir, precision = precision, ext = "-out.maegz", chosen_ligands = biased_ligands, chosen_receptors = chosen_receptors, parallel = None, grid_ext = ".grd", worker_pool=dview)
#dock_ligands_and_receptors(grid_dir, docking_dir, agonist_dir, precision = precision, ext = "-out.maegz", chosen_ligands = agonist_ligands, chosen_receptors = chosen_receptors, parallel = None, grid_ext = ".grd", worker_pool=dview)





25
['/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/clusterer_25clusters_1samples_samples_kdtree/cluster0_sample0.pdb', '/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/clusterer_25clusters_1samples_samples_kdtree/cluster10_sample0.pdb', '/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/clusterer_25clusters_1samples_samples_kdtree/cluster11_sample0.pdb']
Done prepping proteins
/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/grids_clusterer_25clusters_1samples_samples_kdtree
already prepped and mae'd protein
already prepped and mae'd protein
already prepped and mae'd protein
already prepped and mae'd protein
already prepped and m

CompositeError: one or more exceptions from call to method: dock
[Engine Exception]EngineError: Engine 15 died while running task '6d3e4b0f-da2a-4c4c-b980-3d26bb1ef58f'
[Engine Exception]EngineError: Engine 40 died while running task '68932e38-141b-428d-a6bc-cdcac4b5c2cc'

In [22]:
active_dir

'/home/enf/md_simulations/B1AR/reference_receptors/5f8u_hm_P_for_conformation.pdb'

In [11]:
len(dview)

NameError: name 'dview' is not defined

In [ ]:
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *

n_samples = 1
n_clusters = 300

traj_files = get_trajectory_files(traj_dir, traj_ext)
featurized_trajectories = [traj_files[i] for i in feature_indices]
suf_traj_files = [t for t in featurized_trajectories if "suf" in t]
reduced_clusterer = verboseload(clusterer300_clusters)
suf_reduced_clusterer_file = "%s/suf_reduced_clusterer.h5" %tica_dir

suf_reduced_tica_file = "%s/suf_reduced_tica.h5" %tica_dir
suf_reduced_tica = [x[:,range(1,5)] for x in suf_tica]
suf_reseeded_dir = "%s/suf_reseeded_samples" %tica_dir
if not os.path.exists(suf_reseeded_dir):
    os.makedirs(suf_reseeded_dir)
verbosedump(suf_reduced_tica, suf_reduced_tica_file)

suf_reduced_clusters = [reduced_clusterer.labels_[i] for i in suf_trajs]
suf_reduced_clusterer = copy.deepcopy(reduced_clusterer)
suf_reduced_clusterer.labels_ = suf_reduced_clusters
verbosedump(suf_reduced_clusterer, suf_reduced_clusterer_file)
#reseed_from_clusterer(suf_reduced_clusterer_file, "suf", suf_reseeded_dir, suf_reduced_tica_file, suf_traj_files)


samples_dir = "%s/suf_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
reseed_dir = "%s/suf_clusterer_%dclusters_%dsamples_samples_kdtree_reseed" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/suf_clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)
if not os.path.exists(reseed_dir): os.makedirs(reseed_dir)
print(samples_dir)
traj_files = get_trajectory_files(traj_dir, traj_ext)    
sample_from_clusterer(suf_reduced_clusterer_file, suf_reduced_tica_file, [traj_files[i] for i in suf_trajs], 
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=dview, parallel=False, lig_name="LIG", reseed_dir = reseed_dir)

In [21]:
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *

n_samples = 1
n_clusters = 300

traj_files = get_trajectory_files(traj_dir, traj_ext)
receptor_only_trajs = []
gp_trajs = []
gp_traj_files = []
receptor_traj_files = []
for i, traj_file in enumerate(traj_files):
    if "GP" in traj_file:
        gp_trajs.append(i)
        gp_traj_files.append(traj_file)
    else:
        receptor_only_trajs.append(i)
        receptor_traj_files.append(traj_file)

tica_coords = verboseload(projected_features_dir)

loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/phi_psi_chi2_allprot_projected.h5"...


In [20]:

gp_tica = [tica_coords[i] for i in gp_trajs]
receptor_tica = [tica_coords[i] for i in receptor_only_trajs]

gp_clusterer = "%s/gp_clusterer%d" %(tica_dir, n_clusters)
receptor_clusterer_file = "%s/receptor_clusterer%d" %(tica_dir, n_clusters)

receptor_tica_file = "%s/receptor_tica.h5" %tica_dir
reseeded_dir = "%s/receptor_reseeded_samples_%dclusters" %(tica_dir, n_clusters)
if not os.path.exists(reseeded_dir):
    os.makedirs(reseeded_dir)
verbosedump(receptor_tica, receptor_tica_file)

cluster_minikmeans(tica_dir, receptor_tica_file, traj_dir, n_clusters=300, clusterer_dir=receptor_clusterer_file)


reseed_from_clusterer(receptor_clusterer_file, "receptor", reseeded_dir, receptor_tica_file, receptor_traj_files)


samples_dir = "%s/receptor_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
reseed_dir = "%s/receptor_clusterer_%dclusters_%dsamples_samples_kdtree_reseed" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/receptor_clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)
if not os.path.exists(reseed_dir): os.makedirs(reseed_dir)
print(samples_dir)
traj_files = get_trajectory_files(traj_dir, traj_ext)    
sample_from_clusterer(receptor_clusterer_file, receptor_tica_file, receptor_traj_files, 
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=dview, parallel=False, lig_name="NON", reseed_dir = reseed_dir)

loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/phi_psi_chi2_allprot_projected.h5"...
Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/receptor_tica.h5"... (<type 'list'>)
Clustering by KMeans
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/receptor_tica.h5"...
Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/receptor_clusterer300"... (<class 'msmbuilder.cluster.MiniBatchKMeans'>)
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/receptor_clusterer300"...
300
loading "/home/enf/md_simulations/B1AR/spa

In [22]:
gp_tica = [tica_coords[i] for i in gp_trajs]

gp_clusterer_file = "%s/gp_clusterer%d" %(tica_dir, n_clusters)

gp_tica_file = "%s/gp_tica.h5" %tica_dir
reseeded_dir = "%s/gp_reseeded_samples_%dclusters" %(tica_dir, n_clusters)
if not os.path.exists(reseeded_dir):
    os.makedirs(reseeded_dir)
verbosedump(gp_tica, gp_tica_file)

cluster_minikmeans(tica_dir, gp_tica_file, traj_dir, n_clusters=300, clusterer_dir=gp_clusterer_file)


reseed_from_clusterer(gp_clusterer_file, "gp", reseeded_dir, gp_tica_file, gp_traj_files)


samples_dir = "%s/gp_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
reseed_dir = "%s/gp_clusterer_%dclusters_%dsamples_samples_kdtree_reseed" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/gp_clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)
if not os.path.exists(reseed_dir): os.makedirs(reseed_dir)
print(samples_dir)
traj_files = get_trajectory_files(traj_dir, traj_ext)    
sample_from_clusterer(gp_clusterer_file, gp_tica_file, gp_traj_files, 
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=dview, parallel=False, lig_name="NON", reseed_dir = reseed_dir)

Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/gp_tica.h5"... (<type 'list'>)
Clustering by KMeans
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/gp_tica.h5"...
Saving "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/gp_clusterer300"... (<class 'msmbuilder.cluster.MiniBatchKMeans'>)
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/gp_clusterer300"...
300
loading "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components10all_residues_5f8u_hom_under_cutoff6A-CA_regularization_wolf_autoShrinkage0pt01/gp_clusterer300"...
8
10
5
7
1
3
8
2
4
3
2
6
1
8
2
5
4
11
4
4
5
5
3
2
1
3
1
11
5
4
8
2
1
6
5
6
3
11
5
4


In [15]:
len(dview)

43

In [ ]:
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *

n_samples = 1
n_clusters = 300

traj_files = get_trajectory_files(traj_dir, traj_ext)
featurized_trajectories = [traj_files[i] for i in feature_indices]
apo_traj_files = [t for t in featurized_trajectories if "apo" in t]
reduced_clusterer = verboseload(clusterer300_clusters)
apo_reduced_clusterer_file = "%s/apo_reduced_clusterer.h5" %tica_dir

apo_reduced_tica_file = "%s/apo_reduced_tica.h5" %tica_dir
apo_reduced_tica = [x[:,range(1,5)] for x in apo_tica]
apo_reseeded_dir = "%s/apo_reseeded_samples" %tica_dir
if not os.path.exists(apo_reseeded_dir):
    os.makedirs(apo_reseeded_dir)
verbosedump(apo_reduced_tica, apo_reduced_tica_file)

apo_reduced_clusters = [reduced_clusterer.labels_[i] for i in apo_trajs]
apo_reduced_clusterer = copy.deepcopy(reduced_clusterer)
apo_reduced_clusterer.labels_ = apo_reduced_clusters
verbosedump(apo_reduced_clusterer, apo_reduced_clusterer_file)
#reseed_from_clusterer(apo_reduced_clusterer_file, "apo", apo_reseeded_dir, apo_reduced_tica_file, apo_traj_files)


samples_dir = "%s/apo_clusterer_%dclusters_%dsamples_samples_kdtree" %(tica_dir, n_clusters, n_samples)
reseed_dir = "%s/apo_clusterer_%dclusters_%dsamples_samples_kdtree_reseed" %(tica_dir, n_clusters, n_samples)
samples_indices_file = "%s/apo_clusterer_%dclusters_%dsamples_samples_kdtree_indices.h5" %(tica_dir, n_clusters, n_samples)
if not os.path.exists(samples_dir): os.makedirs(samples_dir)
if not os.path.exists(reseed_dir): os.makedirs(reseed_dir)
print(samples_dir)
traj_files = get_trajectory_files(traj_dir, traj_ext)    
sample_from_clusterer(apo_reduced_clusterer_file, apo_reduced_tica_file, [traj_files[i] for i in apo_trajs], 
                      n_samples, samples_dir, samples_indices_file,
                      worker_pool=dview, parallel=False, lig_name="LIG", reseed_dir = reseed_dir)

In [ ]:
import custom_clusterer
reload(custom_clusterer)
from custom_clusterer import *
traj_files = get_trajectory_files(traj_dir, traj_ext)
featurized_trajectories = [traj_files[i] for i in feature_indices]
oxy_traj_files = [t for t in featurized_trajectories if "oxy" in t]
reduced_clusterer = verboseload(clusterer300_clusters)
oxy_reduced_clusterer_file = "%s/oxy_reduced_clusterer.h5" %tica_dir

oxy_reduced_tica_file = "%s/oxy_reduced_tica.h5" %tica_dir
oxy_reduced_tica = [x[:,range(1,10)] for x in oxy_tica]
verbosedump(oxy_reduced_tica, oxy_reduced_tica_file)

oxy_reduced_clusters = [reduced_clusterer.labels_[i] for i in oxy_trajs]
oxy_reduced_clusterer = copy.deepcopy(reduced_clusterer)
oxy_reduced_clusterer.labels_ = oxy_reduced_clusters
verbosedump(oxy_reduced_clusterer, oxy_reduced_clusterer_file)
reseed_from_clusterer(oxy_reduced_clusterer_file, "oxy", tica_dir, oxy_reduced_tica_file, oxy_traj_files)

In [ ]:
closest_indices = find_closest_indices_to_cluster_center(tics_1_3_file, clusterer16_clusters)

In [ ]:
closest_indices

In [ ]:
from scipy.stats import gamma 
x = np.linspace(0.,16.0, 100)
y = gamma.pdf(x, 5.0, loc=8.0, scale=1)
plt.plot(x,y)

In [ ]:
from scipy.stats import gamma 
x = np.linspace(0.,16.0, 100)
y = gamma.pdf(x, 3.0, loc=8.0, scale=0.4)
plt.plot(x,y)
plt.show()

In [ ]:
reseed_from_clusterer(oxy_clusterer_file, "oxy", tica_dir, oxy_tica_file, [trajs[i] for i in oxy_trajs])

In [ ]:
min_pop = [3, 4, 6, 8, 9, 26, 29, 30, 31, 32, 33, 39, 41, 43, 52, 55]


In [ ]:
plot_all_tics_and_clusters(tica_dir, oxy_tica_file, oxy_clusterer_file, lag_time, tic_range=range(0,5), main = "Oxy_Reseeds", label = "cluster_id", active_cluster_ids = min_pop, intermediate_cluster_ids = [], inactive_cluster_ids = [], concatenate=True, axes=tica_axes)